# Testing PyTorch Implementation
This notebook tests the PyTorch implementation of backpropogation and gradient descent to create a simple nerual net that is trained on the MNIST digit dataset. This can be used as a benchmark for comparing against our custom neral net implementation to see differences between using a library vs just coding from scratch.



Set up necessary imports

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.nn.functional as F

Set up training and test data as DataLoaders

In [9]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

Define nerual net model

In [4]:
# Define the neural network model
class FeedForwardNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Model parameters
input_size = 784  # Example: 28x28 pixels for MNIST dataset
hidden_size = 128
num_classes = 10  # Example: Number of classes for MNIST dataset
learning_rate = 0.001
num_epochs = 10

# Create the model, loss function, and optimizer
model = FeedForwardNN(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataloader):
        # Flatten the images and convert to PyTorch tensors
        images = images.view(-1, input_size)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_dataloader)}], Loss: {loss.item():.4f}')

print("Training complete.")

Epoch [1/10], Step [100/938], Loss: 0.4315
Epoch [1/10], Step [200/938], Loss: 0.1787
Epoch [1/10], Step [300/938], Loss: 0.3425
Epoch [1/10], Step [400/938], Loss: 0.3499
Epoch [1/10], Step [500/938], Loss: 0.1738
Epoch [1/10], Step [600/938], Loss: 0.1645
Epoch [1/10], Step [700/938], Loss: 0.1462
Epoch [1/10], Step [800/938], Loss: 0.0835
Epoch [1/10], Step [900/938], Loss: 0.0840
Epoch [2/10], Step [100/938], Loss: 0.1296
Epoch [2/10], Step [200/938], Loss: 0.1725
Epoch [2/10], Step [300/938], Loss: 0.1474
Epoch [2/10], Step [400/938], Loss: 0.0961
Epoch [2/10], Step [500/938], Loss: 0.1313
Epoch [2/10], Step [600/938], Loss: 0.0905
Epoch [2/10], Step [700/938], Loss: 0.1664
Epoch [2/10], Step [800/938], Loss: 0.0629
Epoch [2/10], Step [900/938], Loss: 0.3032
Epoch [3/10], Step [100/938], Loss: 0.2012
Epoch [3/10], Step [200/938], Loss: 0.1245
Epoch [3/10], Step [300/938], Loss: 0.0706
Epoch [3/10], Step [400/938], Loss: 0.0623
Epoch [3/10], Step [500/938], Loss: 0.0468
Epoch [3/10

Set up testing method

In [12]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.view(-1, input_size)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Testing the model

In [14]:
test(model, test_dataloader)


Test set: Average loss: -10.0460, Accuracy: 9786/10000 (98%)

